In [1]:
#Import libraries
import os
import pydicom
import numpy as np
import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

/home/yuan/Software/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Import own utilities
import preproc
import net3d_model as model_3d
import utils
import keraOverlay as overlay

In [3]:
# Load data and masks as dicom files
data_dir = '../../data'                   #Please change the path accordingly before make any changes
normalNames = data_dir + '/GoodData_256auto.txt'
img_dir = data_dir + '/256Auto/images'
msk_dir = data_dir + '/256Auto/masks'


In [4]:
n_slices = 256

model, prediction = model_3d.conv_net(size = 256, activation_type = "sigmoid", n_slices = n_slices, depth = 5, n_base_filters = 16)

In [10]:
#model.summary()

In [11]:
# This visualization might not work depending on how graphviz is installed, but it gives a simple overview of the structure.
#To do: output this image to file or in pop up window
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

<IPython.core.display.SVG object>

In [5]:
model.compile(loss=model_3d.dice_coef_loss, optimizer='adam', metrics=['acc', model_3d.dice_coef])

In [ ]:
model.fit_generator(
    generate_data('~/my_data', batch_size),
    steps_per_epoch=len(os.listdir('~/my_data')) // batch_size)

In [6]:
normal_namelist = []
with open(normalNames) as f:
    content = f.readlines()
    normal_namelist = [x.strip() for x in content] 

In [7]:
#Will take a long time
#history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size = batch_size,
#                   epochs=10, verbose=1, shuffle=True)
history = model.fit_generator(overlay.generate_batch_data(img_dir, msk_dir, normal_namelist, batch_size=1), epochs=5, steps_per_epoch=len(normal_namelist)/2)

Epoch 1/5


In [1]:
import matplotlib.pyplot as plt


In [ ]:
plt.title('Training Progression')
plt.plot(history.epoch, history.history['loss'], label='loss')
plt.plot(history.epoch, history.history['acc'], label='acc')

#plt.plot(history.epoch, history.history[model_3d.dice_coef], label="dice coef")  #TBD
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
n_slice = 64
size = 64

index = 2



#data = utils.padImage(x_list[index])
#data_tobox = preproc.to3dpatches(data, )
predictions = model.predict(x_list[index].reshape(1,n_slice,size,size,1), steps=1)

In [ ]:
slice_index = 34
#Show original, mask, prediction#Show or 
show = plt.figure()
data = show.add_subplot(1,3,1)
data.set_title("data")
#plt.imshow(x_list[0][66])
plt.imshow(x_list[index][slice_index].reshape(size,size))

label = show.add_subplot(1,3,2)
label.set_title("label")
plt.imshow(y_list[index][slice_index].reshape(size,size))

prediction = show.add_subplot(1,3,3)
prediction.set_title("prediction")
plt.imshow(predictions[0][slice_index].reshape(size,size))



In [ ]:
from keras import backend

print(predictions[0][slice_index].shape)

#dice = model_3d.dice_coef(predictions[0],x_list[index])
dice = model_3d.dice_coef(predictions[0],predictions[0])

print(predictions[0][slice_index].dtype)
#dice = dice_coef(y_list[index],predictions[0])
print(dice)
#dice = backend.print_tensor(dice*8, message='y_pred = ')
#backend.print_tensor(dice, message='')
#print(dice)
piece = predictions[0]
piece = piece[piece > 0.9]
dice = model_3d.dice_coef(piece,piece)

backend.eval(dice)
